# Load File

The file helper_functions.py contains all the functions that will do a lot of the work behind the scenes to process our data. First import all the functions from inside this file by running this line of code. 

** If for any reason you need to restart the kernel you will have to rerun this line before proceeding.

In [ ]:
from helper_functions import *

Next we must load our file so that we can work with it. The function load_file will take a .asc file exported from HEKA and load it into a dataframe. A dataframe is simply a python variable that represents tabular data making it easy to work with. HEKA exports have headers separating data throughout the file so the function counts the number of headers interspersed through the file to get a measurement of the number of sweeps then strips them and concatenates the data to make a single table. It will then add a column with the sweep identity and adjust the units to ms, pA, and mm Hg, for time, current, and pressure respectively. All you need to do is type in the path to the file you want to work with. If the file is in the same folder as this code you can just type the filename with the extension.

In [ ]:
dat =load_file('p50_ex.asc')

## Show the first 10 rows of the dataframe
print(dat.head(10))

# Analyzing P50 Curves

## 2. Plot Raw Sweeps

Having loaded our data we can now plot the sweeps. I will use the plotly plotting library since it allows for some basic interaction so you can explore the data a bit. The pressure steps are plotted above the corresponding current traces. This is a similar representation to that you will commonly see in publications without scalebars. The function plot_sweeps() uses the dataframe we loaded in and styles the plot. This is the raw data prior to any preprocessing. It's always a good idea to look at the data before and after preprocessing to make sure the differences make sense. 

In [ ]:
fig = plot_sweeps(dat)
fig.show()

## 3. Preprocessing

If you look at the current traces above there are a few things we may want to do. First, we may want to only look at a subset of the trace. For instance if you have a protocol with a long prepulse before the stimulus we may want to focus in on the stimulus. Second, sometimes you may find you lose your seal in the last sweep so you may want to remove this sweep from the visualization and subsequent analysis. This should be done sparingly and with good reason! Finally, if you hover over the current prior to the stimulus you'll see that it isn't quite at zero. There is always some amount of leak proportional to your seal quality. If this changes substantially between sweeps you may not want to use this particular recording. Otherwise it is important we subtract this baseline from the total current. The baseline is typically quantified as the mean current over a time window where the current appears relatively stable prior to stimulus onset. Importantly, the baseline is calculated for each sweep individually in case there is slight drift. This is performed by the function baseline_subtract() All three of these can be accomplished in the code block below. If you'd like to subset the window or remove traces be sure to uncomment those lines. Baseline subtraction should be done for every recording so I left it uncommented by default. Be sure to comment the lines back out for future analyses where you want them turned off.

In [ ]:
## Removing traces: fill in the list removeList = [sweepnumbers] with sweep numbers that you would like to remove separated by commas.
#removeList = [8]
#dat = dat.query('sweep not in @removeList')

## Baseline subtract: Select a 50 ms window where the baseline appears stable and set the values in baselineWindow = [start, end] with the times in ms.
baselineWindow = [300, 400]
dat = baseline_subtract(dat, baselineWindow)

## Subsetting the data window: Set the two values in subsetWindow = [start, end] to the appropriate values in ms.
subsetWindow = [4990, 5300]
dat = dat.query('ti >= @subsetWindow[0] and ti < @subsetWindow[1]')

fig = plot_sweeps(dat)
fig.show()

Inspect the two plots and make sure you are seeing the expected transformations based on any preprocessing you performed

## 4. Add Scalebars

This part is optional and adds scalebars to the processed traces in case you want to present the data for clearer presentation. This is accomplished by the function add_scalebars(). You just need to fill in the values for the dictionary variable loc that will contain the start and end points for each of the scales. The dictionary is of the form: 
loc = {'t' : [start, end], 'i' : [start,end], 'p' : [start, end]} and should be in units of ms, pA, and mm Hg respectively. If you'd like you can save the image by clicking the camera in the top-right and downloading the plot as a png.

In [ ]:
locs = {'t':[440, 490], 'i':[-80,-60], 'p':[-80, -40]}
fig = add_scalebars(dat, fig, locs)
fig.show()

## 5. Plot P50 Data

In pressure clamp experiments we are often interested in the stimulus-response relationship. Here, the stimulus is an essentially square pressure pulse and the response is often summarized using the peak current amplitude. Peak currents are used in the case of Piezo1 due to the fact that it inactivates. For the study of non-inactivating currents you may see other parameters used in the stimulus-response characterization such as steady-state current or tail currents. The function sweep_summary( ) will take the data, a window over which to summarize the data, and a summary statistic (Max, Min, or Mean). In our case, since Piezo currents are inward at negative potentials we will use the Min. I have made it so the function takes in a window to calculate the summary statistic over for the simple reason that we are naively taking the extrema of all the datapoints fed to the function. This means that if there is a capacitive artifact or some other artifact in the current that leads to a spurious extrema the function will think that is the peak current. By telling the function what window we expect to see this peak over we can avoid this in most cases.

In [ ]:
## Be sure to change the window in analysisWindow = [start, end] to the appropriate times in ms for your analysis.
analysisWindow = [4800, 5100]

## You can select the appropriate summary statistic by typing in either 'Min', 'Max', or 'Mean' as a function argument
summaryDat = sweep_summary(dat, analysisWindow, 'Min')
print(summaryDat)


You should notice in the table above that we have an additional column that was calculated by our function sweep_summary() labeled min_norm_i. While the raw currents are interesting this can vary between patches. For comparison it is often beneficial to normalize this current to the maximum current in the stimulus response curve. This relies on some assumptions that I will discuss shortly after visualizing the data below. The function plot_summary will plot the data. Since the column with our y value of interest will vary based on our summary statistic you just need to manually type in the appropriate yval column label. As long as you are using 'Min" you shouldn't have to change this

In [ ]:
fig2 = plot_summary(summaryDat, yval = 'min_norm_i')
fig2.show()

You should see in the graph that the fractional current (normalized to the largest value across sweeps) shows a sigmoid relationship as a function of stimulus intensity. Recall that the total macroscopic current is a function of $N * P_{o} * i$, where $N$ is the total number of channels in the patch, $P_{o}$ is the open probability, and $i$ is the single channel current at a given voltage. Both N and i should not change as a function of stimulus intensity (although for pressure clamp it is unclear if more membrane is sucked into the pipette at large pressures resulting in a potential change in N). Thus, any change in macroscopic current should be due to a change in the open probability of the channels present in the patch. This relationship can be described by the following sigmoid function:


 <font size="5">$\frac{1}{1 + \exp^\frac{p_{50} - p}{k}}$</font>

Here $p_{50}$ is the pressure of half activation, $p$ is the pressure being applied, and $k$ describes the slope at the inflection point. The appearance of this sigmoid relationship is the consequence of the fact that we have a transition between two states from a closed state to an open state with a probability dependent on the pressure being applied. At low pressures the relative current-pressure relationship will appear roughly exponential. At higher pressures we begin to saturate the pool of channels available in the patch and approach an open probability of 1. For these reasons the normalization to the largest value across sweeps only really makes sense if the fractional current does indeed saturate (or get close) over the stimulus interval.

Given this knowledge we can summarize the stimulus response relationship using the sigmoid function above which is written in the python function sigmoid_fit( ). We then use the scipy library function curve_fit which takes as arguments the fitting function, x values, and y values and returns the fitted parameters (p50 and k) as well as a covariance matrix. Then, using the function fit_layer( ), I can overlay the fit onto the data and you can see that it does indeed describe the stimulus-response relationship fairly well.

In [ ]:
popt, pcov = curve_fit(sigmoid_fit, summaryDat.pressure, summaryDat['min_norm_i'])
fig2 = fit_layer(summaryDat, fig2, popt)
fig2.show()

print(f'P50: {round(popt[0],2)}')
print(f'slope: {round(popt[1],2)}')


We can see the sigmoid fits our data pretty well from the overlayed plot above. Using this the p50 becomes a natural quantification for the threshold for channel gating characteristic for pressure-gated channels. The slope is a measure of sensitivity of channel gating with a more switch-like curve corresponding to a highly sensitive pressure sensor. Be sure to record these parameters during your characterization. It will be a good idea to also save these plots in case you need to look back at the data and the summary data in the table. You can manually save the plots as I mentioned earlier. The following code will save the summary data. Just change the filename to one appropriate for your organizational needs.

In [ ]:
summaryDat.to_csv('2020-09-28_hek293t_hp1_WT_p50_4.csv')

# Analyzing Single-channel Openings

## 1. Visualizing Raw Data

An important intrinsic property of channels is the single channel conductance. Recall that the single channel current of a channel is dependent on the single channel conductance based on the following relationship: 


$i = g(V - E_{m})$ 

Here $i$ is the single-channel current, $g$ is the single-channel conductance, $V$ is the voltage and $E_{m}$ is the equilibrium potential of the channel. Together the terms in the parantheses are the driving force. Mutations that change the single channel conductance can lead to either gain or loss of function by modulating the degree of current flux due to channel opening. It can also give us some insights into biophysical properties of the channel. Typically mutations affecting single-channel conductance tend to be found near the pore of the protein. 

We will be summarizing single-channel properties by quantifying the slope-conductance. This involves measuring the single channel conductance at three different voltages (changing the driving force) and fitting a line to the current as a function of the voltage. Based on the relationship above we expect that this relationship is linear with the slope $g$. Let us begin by visualizing the data. Unfortunately plotly for its use with generating interactive plots does not offer a convenient way to code out the grid elements and axis without hard coding it so I just left them in for this initial visualization. I also initialize an empty dictionary which is just a python object to store key:value pairs that you will populate with current measurements and other identifiers as you go through the analysis.

In [ ]:
roiSummary = {'sweep':[], 'timestamp':[], 'mean_i':[], 'std_i':[], 'v':[]}

fig = plot_sweeps_stacked(dat)
fig.show()

You should be able to see discrete channel openings in the traces above. Sometimes you may see multiple channels opening on top of each other which would which should have a current roughly equivalent to an integer multiple of the single channel current. It is critical that at least on the time-scale that we are looking at the baseline current is stable. If you see a systematic drift in the current don't use that trace and find a higher quality trace to analyze.

For the analysis that follows you should try to find well-isolated single channel openings in the traces above. Avoid short flickers of channel opening. As a rule of thumb try to find clear openings with equal time in the closed state on either side. We will also not be analyzing multiple channel openings here. Compared to p50 analysis, analyzing single-channel conductance in this manner is slightly more labor intensive. Try to zoom in on such an opening above. Once you isolate such an opening use the function isolate_opening( ) to isolate that part of the trace for further analysis. You will need to change the variable sweep to the sweep number that you want to analyze and the window of the form window = [start, end] to the appropriate time in ms surrounding the region of interest. The plot below will show the region you isolated so make sure this matches your expectation.

## 2. Isolating Single Openings

In [ ]:
sweep = 8
window = [5200, 5380]

roi = isolate_opening(dat, sweep, window)
roiFig = plot_sweeps(roi)
roiFig.show()

## 3. Plotting the Frequency Histogram and Estimating i

We can estimate the amplitude of this opening using a frequency histogram. Since we have isolated a single opening we expect to see two peaks in our histogram. The first will reflect occupancy of the non-conducting state with some variance due to electrical noise and the second will reflect the conducting state. In some interesting cases you may find subconductance states between the two. This may reflect some interesting biophysical properties of the channel. For instance, sometimes subconductance states while always present are occupied for extremely short periods of time so they are difficult to observe functionally. Sometimes mutations may change the relative stability of certain protein conformations and stabilize these subconductance states. These would appear as a peak between the closed and fully open state likely of much smaller amplitude. The function frequency_histogram( ) will take the roi you isolated and plot this frequency histogram for you. The function takes three arguments: the dataframe, the number of bins, and the number of gaussians to fit it with. The default is a sum of two gaussians. Currently I have only programmed in 1, 2, or 3 gaussians. In almost all cases you should use 2. You may need to change the number of bins. I have it set at 50 to start off. If you come across evidence for a subconductance state it may be worth trying the triple gaussian.

In [ ]:
fig, popt, pcov = frequency_histogram(roi, 50, 3)
fig.show()

## 4. Appending the Data to a Dictionary

Make sure that the fits above reflect their underlying histograms accurately. Notice that since we did not baseline subtract the closed state is not at zero. This is fine for our case since we are actually interested in the difference between the two means. This difference will reflect the single channel current. The frequency histogram function above returned the parameters for each of the gaussians plotted above in the variables popt (parameters for the gaussians) and pcov (covariance matrix). We will add these values to the dictionary we initialized earlier as well as the sweep number and a timestamp so that you know which opening the measurement is associated with if you need to go back at any point.

In [ ]:
roiSummary['sweep'].append(np.mean(roi.sweep)) 
roiSummary['timestamp'].append(np.mean(roi.ti))
roiSummary['mean_i'].append(round(popt[3] - popt[2], 2))
roiSummary['std_i'].append(round(popt[4] + popt[5], 2))
roiSummary['v'].append(round(np.mean(roi.v) * 1000))

print(f'Estimated i: {roiSummary["mean_i"][-1]}')
print(f'Standard Deviation: {roiSummary["std_i"][-1]}')




Once you've gotten here if there are other openings you need to analyze from the traces you uploaded go back to step 2 and repeat. Once you are finally done you should save this data with a filename that makes the date of the recording and the cell recorded from clear so that somebody can go back and find the relevant opening based on the available information. We will do this by converting our dictionary to a dataframe and saving it as a csv file.

In [ ]:
SummaryDat = pd.DataFrame(roiSummary)
SummaryDat.to_csv('2020-09-22_hek293t_hp1_PKO_SC100_3_analysis.csv')